# Python vs PySpark Commands
*PySpark*


## Creating a Spark Session

In [1]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("PySpark").getOrCreate()
spark

### Create a Spark dataframe

In PySpark you need to create a Spark session first before you do anything. Then the createdataframe is inherent in session.

In [2]:
# initialize list of lists (same as in python)
data = [["tom", 10], ["nick", 15], ["juli", 14]]

# Create the pandas DataFrame
df = spark.createDataFrame(data, ["Name", "Age"])

## Display Dataframe and it's properties

In [3]:
df.show()

+----+---+
|Name|Age|
+----+---+
| tom| 10|
|nick| 15|
|juli| 14|
+----+---+



In [4]:
# This is closer to pandas df.head()
df.toPandas()

,Name,Age
0,tom,10
1,nick,15
2,juli,14


In [11]:
# View column names
# This is the same
df.columns

['Name', 'Age']

In [5]:
# How many rows are in the dataframe
df.count()

3

## Read in data

In [7]:
path = "students.csv"
df = spark.read.csv(path, header=True)
df.toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


## Aggregate Data

This is method is very similar to pandas but you can only do one metric at a time

In [8]:
df.groupBy("gender").agg({"math score": "mean"}).show()

+------+------------------+
|gender|   avg(math score)|
+------+------------------+
|female|63.633204633204635|
|  male| 68.72821576763485|
+------+------------------+



For more than one aggreate... use this

In [42]:
from pyspark.sql import functions as F

df.groupBy("gender").agg(
    F.min("math score"), F.max("math score"), F.avg("math score")
).show()

+------+---------------+---------------+------------------+
|gender|min(math score)|max(math score)|   avg(math score)|
+------+---------------+---------------+------------------+
|female|              0|             99|63.633204633204635|
|  male|            100|             99| 68.72821576763485|
+------+---------------+---------------+------------------+



## Sparks Immutability

Spark DataFrame's are built on top of RDDs which are immutable in nature, hence Data frames are immutable in nature as well.

So if you make a change to a dataframe like adding a column or changing any of the values in the dataframe using the same naming convention, it will generate a new dataframe (with a new unique ID) instead of updating the existing data frame.

In [9]:
# Let's fetch the id of our dataframe we created above
df.rdd.id()

47

In [10]:
# Even if we duplicate the dataframe, the ID remains the same
df2 = df
df2.rdd.id()

47

In [11]:
# It's not until we change the df in some way, that the ID changes
df = df.withColumn("new_col", df["math score"] * 2)
df.rdd.id()

51

## Spark's Lazy Comuptation

What does that mean exactly?

As the name itself indicates its definition, lazy evaluation in Spark means that the execution will not start until it absolutuley HAS to. 

Let's look at an example. 

In [12]:
# These kinds of commands won't actually be run...
df = df.withColumn("new_col", df["math score"] * 2)

In [13]:
# Until we executute a command like this
collect = df.collect()

In [14]:
# Or this
print(df)

DataFrame[gender: string, race/ethnicity: string, parental level of education: string, lunch: string, test preparation course: string, math score: string, reading score: string, writing score: string, new_col: double]


So you can think of Spark like a lazy teenager who doesn't have to clean his room until you come an inspect it :)

The benefit is saving resources and optimizing the Spark cluster overall. 